# Explore here

In [17]:
import pandas as pd

df = pd.read_csv('../data/raw/ds_salaries.csv')

In [18]:
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
...,...,...,...,...,...,...,...,...,...,...,...
3750,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
3751,2021,MI,FT,Principal Data Scientist,151000,USD,151000,US,100,US,L
3752,2020,EN,FT,Data Scientist,105000,USD,105000,US,100,US,S
3753,2020,EN,CT,Business Data Analyst,100000,USD,100000,US,100,US,L


In [23]:
df.drop(['salary','salary_currency'],inplace=True, axis = 1)

KeyError: "['salary', 'salary_currency'] not found in axis"

In [ ]:
df

In [ ]:
df.experience_level.unique()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,12))

plt.bar(x=df.experience_level,height=df.salary_in_usd)

plt.show()

In [ ]:
df.groupby('experience_level')['salary_in_usd'].mean().plot.bar()

In [ ]:
df[df.experience_level == 'EN'].describe()

In [ ]:
dic = {
    'EN':0,
    'MI':1,
    'SE':2,
    'EX':3
}

In [ ]:
df["experience_level"].replace(dic, inplace=True)

In [ ]:
df

In [ ]:
df['employment_type'].unique()

In [ ]:
df.groupby('employment_type')['salary_in_usd'].mean().plot.bar()

In [ ]:
dic_et = {'PT':0,'FL':1,'CT':2,'FT':3}

In [ ]:
df['company_size'].unique()

In [ ]:
df.groupby('company_size')['salary_in_usd'].mean().plot.bar()

In [ ]:
dic_cs = {'S':0,'L':1,'M':2}

In [ ]:
df["employment_type"].replace(dic_et, inplace=True)
df["company_size"].replace(dic_cs, inplace=True)
df

In [ ]:
df.groupby('job_title')['salary_in_usd'].mean().plot.bar()

In [ ]:
df.drop(['job_title'],axis=1,inplace=True)

In [ ]:
df.groupby('employee_residence')['salary_in_usd'].mean().plot.bar()

In [ ]:
df['employee_residence_num'], unique_values = pd.factorize(df['employee_residence'])

In [ ]:
df

In [ ]:
unique_values

In [ ]:
dic_er = dict(enumerate(unique_values))

In [ ]:
dic_er

In [ ]:
dic_er = {valor:codigo for codigo, valor in dic_er.items()}
dic_er

In [ ]:
df['company_location_num'], unique_values = pd.factorize(df['company_location'])
dic_cl = dict(enumerate(unique_values))
dic_cl = {valor:codigo for codigo, valor in dic_cl.items()}
dic_cl

In [ ]:
df

In [ ]:
df.drop(['company_location','employee_residence'],axis=1,inplace=True)

In [ ]:
df

In [ ]:
X = df.drop(['salary_in_usd'],axis=1)
y = df['salary_in_usd']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

X_train

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

model1 = RandomForestRegressor(random_state=42)
model2 = XGBRegressor()

model1.fit(X_train, y_train)
model2.fit(X_train, y_train)

y_pred1 = model1.predict(X_test)
y_pred2 = model2.predict(X_test)

from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score

print(f'rmse modelo 1: {root_mean_squared_error(y_test,y_pred1)} r2: {r2_score(y_test,y_pred1)}')
print(f'rmse modelo 2: {root_mean_squared_error(y_test,y_pred2)} r2: {r2_score(y_test,y_pred2)}')

In [ ]:
params = {
	'max_depth': [3, 4, 5],
	'learning_rate': [0.01, 0.1, 0.2],
	'n_estimators': [20, 50, 100, 200, 300]
}
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(model2, params, scoring='neg_mean_squared_error', cv=5, n_jobs=2)
grid.fit(X_train, y_train)

print(grid.best_params_)
print(grid.best_estimator_)

best_model = grid.best_estimator_
root_mean_squared_error(y_test,best_model.predict(X_test))

In [ ]:
best_model.save_model('../models/xgboost_optimizado.json')

In [ ]:
import json

with open('../data/processed/dic_cl.json','w', encoding='utf-8') as archivo:
    json.dump(dic_cl,archivo,ensure_ascii=False,indent=4)#company_location
with open('../data/processed/dic_el.json','w', encoding='utf-8') as archivo:
    json.dump(dic,archivo,ensure_ascii=False,indent=4)#experience_level
with open('../data/processed/dic_et.json','w', encoding='utf-8') as archivo:
    json.dump(dic_et,archivo,ensure_ascii=False,indent=4)#employment_type
with open('../data/processed/dic_er.json','w', encoding='utf-8') as archivo:
    json.dump(dic_er,archivo,ensure_ascii=False,indent=4)#employee_residence
with open('../data/processed/dic_cs.json','w', encoding='utf-8') as archivo:
    json.dump(dic_cs,archivo,ensure_ascii=False,indent=4) #company_size

In [ ]:
X_train.columns